# Proyecto Nanotecnología.  

En el campo de la nanotecnología, los científicos realizan experimentos sobre materiales que operan a nano escala, estos generan datos relacionados con las propiedades físicas y químicas de los materiales estudiados.

Estos datos son importantes para comprender y predecir cómo los materiales se comportan al aplicar ciertas fuerzas o condiciones, como cambios de temperatura o presión. Para poder usarlos y poder realizar predicciones precisas, es necesario el construir una estructura de datos que permita el acceso de esa información de forma optimizada.

##Importamos los datos.

In [0]:
# creación de las carpetas "bronce", "silver", "gold". 

path = '/FileStore/tables'
dbutils.fs.mkdirs(path + "/bronce")
dbutils.fs.mkdirs(path + "/silver")
dbutils.fs.mkdirs(path + "/gold")

Out[59]: True

In [0]:
#Declaración de las variables src y dest
src1 = "dbfs:/FileStore/Materiales.csv"
dest1 = "dbfs:/FileStore/tables/bronce/Materiales.csv"

src2 = "dbfs:/FileStore/Condiciones.csv"
dest2 = "dbfs:/FileStore/tables/bronce/Condiciones.csv"

src3 = "dbfs:/FileStore/Propiedades.csv"
dest3 = "dbfs:/FileStore/tables/bronce/Propiedades.csv"


In [0]:
# Importación de las bibliotecas necesarias.
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Creación de una instancia de SparkSession.
spark = SparkSession.builder.appName("CargaCSV").getOrCreate()

# Carga de los archivos CSV dentro de DataFrames.
df1 = spark.read.csv(src1, header=True, inferSchema=True)
df2 = spark.read.csv(src2, header=True, inferSchema=True)
df3 = spark.read.csv(src3, header=True, inferSchema=True)


In [0]:
#Observación de los datos.

df1.show(5) 
df2.show(5)  
df3.show(5) 

+-----------+--------------------+--------------------+--------------------+--------------+
|id_material|                name|   compuesto_quimico|          aplicacion|fecha_creacion|
+-----------+--------------------+--------------------+--------------------+--------------+
|          1|Nanopartículas de...|Nanopartículas de...|Revestimientos ce...|    2024-04-26|
|          2|Nanopartículas de...|Nanopartículas de...|              Imanes|    2024-02-02|
|          3|Nanopartículas de...|       MXenes (null)|Materiales de con...|    2024-07-05|
|          4|          Fullerenos|Nanopartículas de...|              Imanes|    2024-06-22|
|          5|Nanomateriales cr...|Nanopartículas de...|Pantallas de alta...|    2023-11-22|
+-----------+--------------------+--------------------+--------------------+--------------+
only showing top 5 rows

+--------------+-----------+------------------+----------------+--------------------+-----------------+
|id_experimento|id_material|temperatura_pru

##Limpieza de los datos

In [0]:
#unión de los tres DataFrames.   

df_union = df1.join(df2, "id_material").join(df3, "id_material")

In [0]:
#Datos repetidos por identificadores unicos. 

df_duplicates = df_union.groupBy("id_material", "id_experimento", "id_propiedad").count().filter("count > 1")
df_duplicates.show()


+-----------+--------------+------------+-----+
|id_material|id_experimento|id_propiedad|count|
+-----------+--------------+------------+-----+
+-----------+--------------+------------+-----+



In [0]:
#Datos duplicados por condiciones del Experimento.

df_duplicates = df_union.groupBy("id_material", "temperatura_prueba", "presion_aplicada", "duracion_experimento", "fecha_experimento").count().filter("count > 1")

df_duplicates.show() #Mostramos si existen datos duplicados.

+-----------+------------------+----------------+--------------------+-----------------+-----+
|id_material|temperatura_prueba|presion_aplicada|duracion_experimento|fecha_experimento|count|
+-----------+------------------+----------------+--------------------+-----------------+-----+
+-----------+------------------+----------------+--------------------+-----------------+-----+



In [0]:
#Datos duplicados por las propiedades del Material

df_duplicates = df_union.groupBy("id_material", "resistencia", "elasticidad", "conductividad_termica", "densidad", "diametro_nanoparticulas", "espesor_capa", "capacidad_absorcion_luz").count().filter("count > 1")

df_duplicates.show() #Mostramos si existen datos duplicados.

+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+
|id_material|resistencia|elasticidad|conductividad_termica|densidad|diametro_nanoparticulas|espesor_capa|capacidad_absorcion_luz|count|
+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+
+-----------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+-----+



In [0]:
# Contamos aquellos valores nulos en cada columna de df_union.
from pyspark.sql.functions import col, sum

df_union.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_union.columns]).show()

+-----------+----+-----------------+----------+--------------+--------------+------------------+----------------+--------------------+-----------------+------------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+
|id_material|name|compuesto_quimico|aplicacion|fecha_creacion|id_experimento|temperatura_prueba|presion_aplicada|duracion_experimento|fecha_experimento|id_propiedad|resistencia|elasticidad|conductividad_termica|densidad|diametro_nanoparticulas|espesor_capa|capacidad_absorcion_luz|
+-----------+----+-----------------+----------+--------------+--------------+------------------+----------------+--------------------+-----------------+------------+-----------+-----------+---------------------+--------+-----------------------+------------+-----------------------+
|          0|   0|                0|         0|             0|             0|                 0|               0|                   0|                0|  

In [0]:
#Cambiamos el nombre de la columna name a nombre. 

df_renamed = df_union.withColumnRenamed("name", "nombre")
df_renamed.describe() #Mostramos las columnas del dataframe. 

Out[69]: DataFrame[summary: string, id_material: string, nombre: string, compuesto_quimico: string, aplicacion: string, id_experimento: string, temperatura_prueba: string, presion_aplicada: string, duracion_experimento: string, id_propiedad: string, resistencia: string, elasticidad: string, conductividad_termica: string, densidad: string, diametro_nanoparticulas: string, espesor_capa: string, capacidad_absorcion_luz: string]

In [0]:
#Nos aseguramos de que nuestros datos tengan el tipo de dato correcto. 

from pyspark.sql.functions import col

df_corrected = (df_renamed
                .withColumn("id_material", col("id_material").cast("integer"))
                .withColumn("temperatura_prueba", col("temperatura_prueba").cast("integer"))
                .withColumn("presion_aplicada", col("presion_aplicada").cast("double"))
                .withColumn("duracion_experimento", col("duracion_experimento").cast("integer"))
                .withColumn("resistencia", col("resistencia").cast("integer"))
                .withColumn("elasticidad", col("elasticidad").cast("integer"))
                .withColumn("conductividad_termica", col("conductividad_termica").cast("double"))
                .withColumn("densidad", col("densidad").cast("double"))
                .withColumn("diametro_nanoparticulas", col("diametro_nanoparticulas").cast("integer"))
                .withColumn("espesor_capa", col("espesor_capa").cast("integer"))
                .withColumn("capacidad_absorcion_luz", col("capacidad_absorcion_luz").cast("integer"))
               )

df_corrected.printSchema() #Nos aseguramos que el tipo de datos sea el adecuado. 


root
 |-- id_material: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- compuesto_quimico: string (nullable = true)
 |-- aplicacion: string (nullable = true)
 |-- fecha_creacion: date (nullable = true)
 |-- id_experimento: integer (nullable = true)
 |-- temperatura_prueba: integer (nullable = true)
 |-- presion_aplicada: double (nullable = true)
 |-- duracion_experimento: integer (nullable = true)
 |-- fecha_experimento: date (nullable = true)
 |-- id_propiedad: integer (nullable = true)
 |-- resistencia: integer (nullable = true)
 |-- elasticidad: integer (nullable = true)
 |-- conductividad_termica: double (nullable = true)
 |-- densidad: double (nullable = true)
 |-- diametro_nanoparticulas: integer (nullable = true)
 |-- espesor_capa: integer (nullable = true)
 |-- capacidad_absorcion_luz: integer (nullable = true)



In [0]:
# Normalización de las columnas numéricas.

from pyspark.ml.feature import VectorAssembler, StandardScaler

# Seleccionamos las columnas numéricas para la normalización.
numeric_cols = ["temperatura_prueba", "presion_aplicada", "duracion_experimento", "resistencia", "elasticidad", "conductividad_termica", "densidad", "diametro_nanoparticulas", "espesor_capa", "capacidad_absorcion_luz"]

# Creamos un "VectorAssembler" para combinar las columnas numéricas en una sola columna de características
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
# Transformación del DataFrame
df_features = assembler.transform(df_corrected)

# Creamos un StandardScaler para normalizar las características
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

# Ajustamos y se transforman los datos
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

# Se muestran las primeras filas para verificar la normalización
df_scaled.select("scaled_features").show(5)

+--------------------+
|     scaled_features|
+--------------------+
|[1.28063416017186...|
|[-0.7521184750215...|
|[0.26425784257514...|
|[0.60304994844071...|
|[0.60304994844071...|
+--------------------+
only showing top 5 rows



In [0]:
#Descomponemos el Vector de Características Normalizadas.
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType

# UDF para convertir el vector en una lista.
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))

# Convertimos la columna 'scaled_features' en una lista.
df_array = df_scaled.withColumn("scaled_features_array", vector_to_array(col("scaled_features")))

# Seleccionar las columnas originales y las características normalizadas.
df_final = df_array.select(["id_material", "nombre", "compuesto_quimico", "aplicacion", "fecha_creacion", "id_experimento", "id_propiedad"] + [col("scaled_features_array")[i].alias(numeric_cols[i]) for i in range(len(numeric_cols))])

# Mostrar las primeras filas para verificar.
df_final.show(5)

+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+-------------------+-------------------+--------------------+--------------------+-------------------+---------------------+--------------------+-----------------------+--------------------+-----------------------+
|id_material|              nombre|   compuesto_quimico|          aplicacion|fecha_creacion|id_experimento|id_propiedad| temperatura_prueba|   presion_aplicada|duracion_experimento|         resistencia|        elasticidad|conductividad_termica|            densidad|diametro_nanoparticulas|        espesor_capa|capacidad_absorcion_luz|
+-----------+--------------------+--------------------+--------------------+--------------+--------------+------------+-------------------+-------------------+--------------------+--------------------+-------------------+---------------------+--------------------+-----------------------+--------------------+-----------------------

In [0]:
# Verificamos las estadísticas descriptivas.
df_final.select(numeric_cols).describe().show()


+-------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+-----------------------+--------------------+-----------------------+
|summary|  temperatura_prueba|    presion_aplicada|duracion_experimento|         resistencia|         elasticidad|conductividad_termica|            densidad|diametro_nanoparticulas|        espesor_capa|capacidad_absorcion_luz|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+-----------------------+--------------------+-----------------------+
|  count|                1000|                1000|                1000|                1000|                1000|                 1000|                1000|                   1000|                1000|                   1000|
|   mean|9.321432514752815...|-5.19545517718711...|-1.10156328503308...|7.23110460398857E-15